In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import time
import requests
from api_keys import omdb_api_key

In [3]:
title_file = Path("Resources/title.basics.tsv")
rating_file = Path("Resources/title.ratings.tsv")

title_data = pd.read_csv(title_file, sep='\t')
rating_data = pd.read_csv(rating_file, sep='\t')

# Combine the data into a single DataFrame
combined_df = pd.merge(title_data, rating_data, on="tconst", how="inner")

C:\Users\duvoe\AppData\Local\Temp\ipykernel_2208\2272656127.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_data = pd.read_csv(title_file, sep='\t')


In [4]:
#rename columns
combined_df.rename(columns={"tconst": "IMDB_ID", "titleType": "Type", "runtimeMinutes": "Runtime (Min)", "genres": "Genres", "averageRating": "Avg Rating", "primaryTitle":"Primary Title", "originalTitle": "Original Title"}, inplace=True)

combined_df.head()

,IMDB_ID,Type,Primary Title,Original Title,isAdult,startYear,endYear,Runtime (Min),Genres,Avg Rating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,2100
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.6,282
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance",6.5,2118
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.4,182
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2850


In [7]:
combined_df.count()

IMDB_ID           1497921
Type              1497921
Primary Title     1497920
Original Title    1497920
isAdult           1497921
startYear         1497921
endYear           1497921
Runtime (Min)     1497921
Genres            1497918
Avg Rating        1497921
numVotes          1497921
dtype: int64

In [9]:
combined_df=combined_df.dropna(how="any")
combined_df.count()

IMDB_ID           1497917
Type              1497917
Primary Title     1497917
Original Title    1497917
isAdult           1497917
startYear         1497917
endYear           1497917
Runtime (Min)     1497917
Genres            1497917
Avg Rating        1497917
numVotes          1497917
dtype: int64

In [11]:
combined_df = combined_df.loc[(combined_df["startYear"] != '\\N'), :]
combined_df.count()

IMDB_ID           1497671
Type              1497671
Primary Title     1497671
Original Title    1497671
isAdult           1497671
startYear         1497671
endYear           1497671
Runtime (Min)     1497671
Genres            1497671
Avg Rating        1497671
numVotes          1497671
dtype: int64

In [13]:
# Convert column from string to integer
combined_df['startYear'] = combined_df['startYear'].astype(int)


In [15]:
# Filter movies with years from 2010-2019
years_df = combined_df.loc[(combined_df["startYear"] >= 2010) & (combined_df["startYear"] <= 2019), :]
years_df.head()

,IMDB_ID,Type,Primary Title,Original Title,isAdult,startYear,endYear,Runtime (Min),Genres,Avg Rating,numVotes
31444,tt0050396,short,Final Curtain,Final Curtain,0,2012,\N,22,"Horror,Short",4.5,246
40260,tt0060361,short,EMS nr 1,EMS nr 1,0,2016,\N,14,Short,6.5,6
44306,tt0065047,short,The Sun's Gonna Shine,The Sun's Gonna Shine,0,2014,\N,10,"Documentary,Music,Short",7.0,185
46705,tt0067758,movie,"Simón, contamos contigo","Simón, contamos contigo",0,2015,\N,81,"Comedy,Drama",4.2,47
47755,tt0068943,short,Between Wroclaw and Zielona Góra,Miedzy Wroclawiem a Zielona Góra,0,2010,\N,11,"Documentary,Short",5.3,84


In [17]:
# drop '\\N'
years_df = years_df.loc[(years_df["Runtime (Min)"] != '\\N'), :]
years_df.count()

IMDB_ID           378526
Type              378526
Primary Title     378526
Original Title    378526
isAdult           378526
startYear         378526
endYear           378526
Runtime (Min)     378526
Genres            378526
Avg Rating        378526
numVotes          378526
dtype: int64

In [19]:
# Convert to int
years_df['Runtime (Min)'] = years_df['Runtime (Min)'].astype(int)

In [21]:
# Filter rows for runtime of 60min - 240min
# Can narrow down or widen
# Gets rid of short films
years_runtime_df = years_df.loc[(years_df["Runtime (Min)"] >= 60) & (years_df["Runtime (Min)"] <= 240), :]
years_runtime_df.head()

,IMDB_ID,Type,Primary Title,Original Title,isAdult,startYear,endYear,Runtime (Min),Genres,Avg Rating,numVotes
46705,tt0067758,movie,"Simón, contamos contigo","Simón, contamos contigo",0,2015,\N,81,"Comedy,Drama",4.2,47
47850,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,6.7,8183
75012,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,0,2017,\N,80,"Comedy,Drama,Fantasy",6.5,370
81587,tt0108116,movie,Shepherd on the Rock,Shepherd on the Rock,0,2014,\N,90,Drama,5.8,99
88844,tt0116991,movie,Mariette in Ecstasy,Mariette in Ecstasy,0,2019,\N,101,Drama,7.1,81


In [23]:
years_runtime_df.count()

IMDB_ID           117450
Type              117450
Primary Title     117450
Original Title    117450
isAdult           117450
startYear         117450
endYear           117450
Runtime (Min)     117450
Genres            117450
Avg Rating        117450
numVotes          117450
dtype: int64

In [25]:
## Lots of non-movie titles
years_runtime_df['Type'].value_counts()

Type
movie           80317
tvEpisode       16228
tvMovie          7686
video            5659
tvSeries         4547
tvSpecial        1817
tvMiniSeries     1170
videoGame          16
short              10
Name: count, dtype: int64

In [27]:
# Filters only by movie
years_runtime_type_df = years_runtime_df.loc[(years_runtime_df["Type"] == 'movie'), :]
years_runtime_type_df.count()

IMDB_ID           80317
Type              80317
Primary Title     80317
Original Title    80317
isAdult           80317
startYear         80317
endYear           80317
Runtime (Min)     80317
Genres            80317
Avg Rating        80317
numVotes          80317
dtype: int64

In [29]:
years_runtime_type_df["numVotes"].median()


93.0

In [31]:
votes_df = years_runtime_type_df.loc[years_runtime_type_df["numVotes"] >=1000, :]
votes_df.count()

IMDB_ID           13892
Type              13892
Primary Title     13892
Original Title    13892
isAdult           13892
startYear         13892
endYear           13892
Runtime (Min)     13892
Genres            13892
Avg Rating        13892
numVotes          13892
dtype: int64

In [33]:
sorted_votes = votes_df.sort_values(by="numVotes", ascending=True)
sorted_votes.head(25)

,IMDB_ID,Type,Primary Title,Original Title,isAdult,startYear,endYear,Runtime (Min),Genres,Avg Rating,numVotes
1023868,tt2735292,movie,L.A. Slasher,L.A. Slasher,0,2015,\N,84,"Comedy,Crime,Drama",3.2,1000
1287304,tt5618232,movie,FML,FML,0,2016,\N,98,Comedy,4.8,1000
862976,tt1918806,movie,Entrance,Entrance,0,2012,\N,83,"Drama,Horror,Thriller",4.7,1000
890060,tt2043801,movie,The Bunnyman Massacre,The Bunnyman Massacre,0,2014,\N,90,Horror,3.9,1001
506351,tt10703752,movie,Balloon,Qi qiu,0,2019,\N,102,Drama,6.9,1001
967431,tt2361846,movie,Cheatin',Cheatin',0,2013,\N,76,"Animation,Comedy,Drama",7.0,1001
1454259,tt8804284,movie,American Dharma,American Dharma,0,2018,\N,95,"Biography,Documentary",7.0,1001
1213782,tt4463762,movie,Brasserie Valentine,Brasserie Valentijn,0,2016,\N,102,Romance,5.4,1001
1098193,tt3130734,movie,Phantom Halo,Phantom Halo,0,2014,\N,87,"Crime,Drama,Thriller",5.1,1001
1297825,tt5785200,movie,Machine,Machine,0,2017,\N,148,"Action,Mystery,Romance",2.8,1001


In [35]:
sorted_years_runtime_type_df = sorted_votes.sort_values(by="Avg Rating", ascending=False)
sorted_years_runtime_type_df.head()

,IMDB_ID,Type,Primary Title,Original Title,isAdult,startYear,endYear,Runtime (Min),Genres,Avg Rating,numVotes
1279638,tt5510934,movie,The Unnamed,Oggatonama,0,2016,\N,100,Drama,9.0,5899
1297045,tt5773402,movie,Dominion,Dominion,0,2018,\N,120,"Documentary,Horror",8.9,4708
1481841,tt9448868,movie,BTS World Tour: Love Yourself in Seoul,BTS World Tour: Love Yourself in Seoul,0,2018,\N,112,"Documentary,Music",8.9,2273
1271244,tt5354160,movie,Mirror Game,Aynabaji,0,2016,\N,147,"Crime,Mystery,Thriller",8.9,30104
1259119,tt5190958,movie,Katyar Kaljat Ghusali,Katyar Kaljat Ghusali,0,2015,\N,162,Musical,8.8,2611


In [37]:
# Gets list titles with all criteria
df_final = sorted_years_runtime_type_df
df_final = df_final['Primary Title']
df_final

1279638                               The Unnamed
1297045                                  Dominion
1481841    BTS World Tour: Love Yourself in Seoul
1271244                               Mirror Game
1259119                     Katyar Kaljat Ghusali
                            ...                  
1437458                            Cumali Ceber 2
1098642                           Elment az öszöd
1473819       The Crimean Bridge. Made with Love!
1381545            Cumali Ceber: Allah Seni Alsin
1311330                                      Reis
Name: Primary Title, Length: 13892, dtype: object

In [39]:
len(df_final)

13892

In [41]:
#url = "http://www.omdbapi.com/?apikey=" + omdb_api_key + "&t="

#movies = sorted_years_runtime_type_df["Primary Title"]

#responses = []

#for movie in movies:
    #movie_data = requests.get(url + movie).json()
    #responses.append(movie_data)

In [43]:
#movie_df.to_csv("Output/omdb_api.csv", encoding="utf-8", index=False, header=True)

In [45]:
movie_file = Path("Resources/omdb_api.csv")

movie_df = pd.read_csv(movie_file, encoding="utf-8")

In [47]:
movie_df = pd.DataFrame(movie_df)
movie_df.head(10)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons,Error
0,The Unnamed,2016,NaN,19 Aug 2016,100 min,Drama,Tauquir Ahmed,Tauquir Ahmed,"Shahiduzzaman Selim, Ashraful Tushar, Mosharra...",The coffin of an expatriate worker with manipu...,...,"5,761",tt5510934,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
1,Dominion,2014–2015,TV-MA,19 Jun 2014,31S min,"Action, Drama, Fantasy",NaN,Vaun Wilmott,"Christopher Egan, Tom Wisdom, Roxanne McKee",Follows the perilous journey of a rebellious y...,...,"24,335",tt3079768,series,NaN,NaN,NaN,NaN,True,2.0,NaN
2,BTS World Tour: Love Yourself in Seoul,2018,NaN,26 Jan 2019,112 min,"Documentary, Music",NaN,NaN,"BTS, j-hope, Jimin",A concert film by South Korean group BTS. The ...,...,"2,170",tt9448868,movie,NaN,"$3,509,917",NaN,NaN,True,NaN,NaN
3,Mirror Game,2016,NaN,30 Sep 2016,147 min,"Crime, Mystery, Thriller",Amitabh Reza Chowdhury,"Syed Gaosul Alam Shaon, Anam Biswas, Adnan Adi...","Chanchal Chowdhury, Masuma Rahman Nabila, Part...",A struggling actor becomes a professional impo...,...,"30,065",tt5354160,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
4,Katyar Kaljat Ghusali,2015,NaN,12 Nov 2015,162 min,Musical,Subodh Bhave,"Swarada Burse, Purushottam Darvekar, Prakash K...","Sachin Pilgaonkar, Shankar Mahadevan, Subodh B...","A clash of two musical families, where in the ...",...,"2,604",tt5190958,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
5,Les Misérables in Concert: The 25th Anniversary,2010,Not Rated,03 Oct 2010,178 min,"Drama, Musical","Nick Morris, Laurence Connor, James Powell","Alain Boublil, Claude-Michel Schönberg, Victor...","Alfie Boe, Norm Lewis, Matt Lucas",A celebratory 25th anniversary concert perform...,...,"6,529",tt1754109,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
6,The Weight of Chains 2,2014,NaN,01 Nov 2014,124 min,"Documentary, Comedy, History",Boris Malagurski,Boris Malagurski,"Boris Malagurski, Oliver Stone, Noam Chomsky",'The Weight of Chains 2' is a documentary film...,...,"1,453",tt4733046,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
7,Inception,2010,PG-13,16 Jul 2010,148 min,"Action, Adventure, Sci-Fi",Christopher Nolan,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...,...,"2,602,766",tt1375666,movie,NaN,"$292,587,330",NaN,NaN,True,NaN,NaN
8,Through the Needle,2014,NaN,14 Feb 2014,74 min,"Documentary, Biography, Drama",Arturs Lapinskis,NaN,Edijs Vitolins,Biographical story about a 23 year old guy who...,...,"1,212",tt3557258,movie,NaN,NaN,NaN,NaN,True,NaN,NaN
9,Chaal Jeevi Laiye,2019,NaN,15 Mar 2019,137 min,Drama,Vipul Mehta,"Jainesh Ejardar, Vipul Mehta","Siddharth Randeria, Yash Soni, Aarohi Patel","In his quest to become successful, Aditya Pari...",...,"1,436",tt9610262,movie,NaN,NaN,NaN,NaN,True,NaN,NaN


In [49]:
column_drop_movie_df = movie_df.drop(["Website", "totalSeasons", "DVD"], axis='columns')
column_drop_movie_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error
0,The Unnamed,2016,NaN,19 Aug 2016,100 min,Drama,Tauquir Ahmed,Tauquir Ahmed,"Shahiduzzaman Selim, Ashraful Tushar, Mosharra...",The coffin of an expatriate worker with manipu...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,9.0,"5,761",tt5510934,movie,NaN,NaN,True,NaN
1,Dominion,2014–2015,TV-MA,19 Jun 2014,31S min,"Action, Drama, Fantasy",NaN,Vaun Wilmott,"Christopher Egan, Tom Wisdom, Roxanne McKee",Follows the perilous journey of a rebellious y...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.7,"24,335",tt3079768,series,NaN,NaN,True,NaN
2,BTS World Tour: Love Yourself in Seoul,2018,NaN,26 Jan 2019,112 min,"Documentary, Music",NaN,NaN,"BTS, j-hope, Jimin",A concert film by South Korean group BTS. The ...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.9,"2,170",tt9448868,movie,"$3,509,917",NaN,True,NaN
3,Mirror Game,2016,NaN,30 Sep 2016,147 min,"Crime, Mystery, Thriller",Amitabh Reza Chowdhury,"Syed Gaosul Alam Shaon, Anam Biswas, Adnan Adi...","Chanchal Chowdhury, Masuma Rahman Nabila, Part...",A struggling actor becomes a professional impo...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.9,"30,065",tt5354160,movie,NaN,NaN,True,NaN
4,Katyar Kaljat Ghusali,2015,NaN,12 Nov 2015,162 min,Musical,Subodh Bhave,"Swarada Burse, Purushottam Darvekar, Prakash K...","Sachin Pilgaonkar, Shankar Mahadevan, Subodh B...","A clash of two musical families, where in the ...",...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.8,"2,604",tt5190958,movie,NaN,NaN,True,NaN


In [51]:
only_movies_df = column_drop_movie_df.loc[(column_drop_movie_df["Type"] == 'movie'), :]
only_movies_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error
0,The Unnamed,2016,NaN,19 Aug 2016,100 min,Drama,Tauquir Ahmed,Tauquir Ahmed,"Shahiduzzaman Selim, Ashraful Tushar, Mosharra...",The coffin of an expatriate worker with manipu...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,9.0,"5,761",tt5510934,movie,NaN,NaN,True,NaN
2,BTS World Tour: Love Yourself in Seoul,2018,NaN,26 Jan 2019,112 min,"Documentary, Music",NaN,NaN,"BTS, j-hope, Jimin",A concert film by South Korean group BTS. The ...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.9,"2,170",tt9448868,movie,"$3,509,917",NaN,True,NaN
3,Mirror Game,2016,NaN,30 Sep 2016,147 min,"Crime, Mystery, Thriller",Amitabh Reza Chowdhury,"Syed Gaosul Alam Shaon, Anam Biswas, Adnan Adi...","Chanchal Chowdhury, Masuma Rahman Nabila, Part...",A struggling actor becomes a professional impo...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.9,"30,065",tt5354160,movie,NaN,NaN,True,NaN
4,Katyar Kaljat Ghusali,2015,NaN,12 Nov 2015,162 min,Musical,Subodh Bhave,"Swarada Burse, Purushottam Darvekar, Prakash K...","Sachin Pilgaonkar, Shankar Mahadevan, Subodh B...","A clash of two musical families, where in the ...",...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.8,"2,604",tt5190958,movie,NaN,NaN,True,NaN
5,Les Misérables in Concert: The 25th Anniversary,2010,Not Rated,03 Oct 2010,178 min,"Drama, Musical","Nick Morris, Laurence Connor, James Powell","Alain Boublil, Claude-Michel Schönberg, Victor...","Alfie Boe, Norm Lewis, Matt Lucas",A celebratory 25th anniversary concert perform...,...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.8,"6,529",tt1754109,movie,NaN,NaN,True,NaN


In [53]:
empty_metascore = only_movies_df["Metascore"].fillna(value=0)

In [65]:
sorted_scores = only_movies_df.sort_values(by="Metascore", ascending=False)
sorted_scores.head(25)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,"[{'Source': 'Internet Movie Database', 'Value'...",100.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,"[{'Source': 'Internet Movie Database', 'Value'...",99.0,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN
6349,Pinocchio,1940,Approved,23 Feb 1940,88 min,"Animation, Adventure, Comedy","Norman Ferguson, T. Hee, Wilfred Jackson","Carlo Collodi, Ted Sears, Otto Englander","Dickie Jones, Christian Rub, Mel Blanc","A living puppet, with the help of a cricket as...",...,"[{'Source': 'Internet Movie Database', 'Value'...",99.0,7.5,"163,151",tt0032910,movie,"$84,254,167",NaN,True,NaN
9479,Some Like It Hot,1959,Approved,19 Mar 1959,121 min,"Comedy, Music, Romance",Billy Wilder,"Billy Wilder, I.A.L. Diamond, Robert Thoeren","Marilyn Monroe, Tony Curtis, Jack Lemmon","After two male musicians witness a mob hit, th...",...,"[{'Source': 'Internet Movie Database', 'Value'...",98.0,8.2,"288,169",tt0053291,movie,NaN,NaN,True,NaN
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN
6445,Dumbo,1941,G,31 Oct 1941,64 min,"Animation, Adventure, Drama","Samuel Armstrong, Norman Ferguson, Wilfred Jac...","Joe Grant, Dick Huemer, Otto Englander","Sterling Holloway, Edward Brophy, Herman Bing","Ridiculed because of his enormous ears, a youn...",...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,7.2,"144,237",tt0033563,movie,NaN,NaN,True,NaN
595,Manchester by the Sea,2016,R,16 Dec 2016,137 min,Drama,Kenneth Lonergan,Kenneth Lonergan,"Casey Affleck, Michelle Williams, Kyle Chandler",A depressed uncle is asked to take care of his...,...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,7.8,"321,828",tt4034228,movie,"$47,695,371",NaN,True,NaN
822,Gravity,2013,PG-13,04 Oct 2013,91 min,"Drama, Sci-Fi, Thriller",Alfonso Cuarón,"Alfonso Cuarón, Jonás Cuarón","Sandra Bullock, George Clooney, Ed Harris","Dr Ryan Stone, an engineer on her first time o...",...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,7.7,"872,078",tt1454468,movie,"$274,092,705",NaN,True,NaN
51,Parasite,2019,R,08 Nov 2019,132 min,"Drama, Thriller",Bong Joon Ho,"Bong Joon Ho, Han Jin-won","Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong",Greed and class discrimination threaten the ne...,...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,8.5,"999,122",tt6751668,movie,"$53,369,749",NaN,True,NaN
290,12 Years a Slave,2013,R,08 Nov 2013,134 min,"Biography, Drama, History",Steve McQueen,"John Ridley, Solomon Northup","Chiwetel Ejiofor, Michael Kenneth Williams, Mi...","In the antebellum United States, Solomon North...",...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,8.1,"751,913",tt2024544,movie,"$56,671,993",NaN,True,NaN


In [69]:
sorted_scores["Metascore"] = sorted_scores["Metascore"]/10
sorted_scores.head()


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,"[{'Source': 'Internet Movie Database', 'Value'...",10.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,"[{'Source': 'Internet Movie Database', 'Value'...",9.9,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN
6349,Pinocchio,1940,Approved,23 Feb 1940,88 min,"Animation, Adventure, Comedy","Norman Ferguson, T. Hee, Wilfred Jackson","Carlo Collodi, Ted Sears, Otto Englander","Dickie Jones, Christian Rub, Mel Blanc","A living puppet, with the help of a cricket as...",...,"[{'Source': 'Internet Movie Database', 'Value'...",9.9,7.5,"163,151",tt0032910,movie,"$84,254,167",NaN,True,NaN
9479,Some Like It Hot,1959,Approved,19 Mar 1959,121 min,"Comedy, Music, Romance",Billy Wilder,"Billy Wilder, I.A.L. Diamond, Robert Thoeren","Marilyn Monroe, Tony Curtis, Jack Lemmon","After two male musicians witness a mob hit, th...",...,"[{'Source': 'Internet Movie Database', 'Value'...",9.8,8.2,"288,169",tt0053291,movie,NaN,NaN,True,NaN
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,"[{'Source': 'Internet Movie Database', 'Value'...",9.6,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN


In [71]:
sorted_scores["Year"] = sorted_scores["Year"].astype(int)

In [73]:
cleaned_scores_df = sorted_scores.loc[(sorted_scores["Year"] >= 2010) & (sorted_scores["Year"] <= 2019), :]
cleaned_scores_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,"[{'Source': 'Internet Movie Database', 'Value'...",10.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,"[{'Source': 'Internet Movie Database', 'Value'...",9.9,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,"[{'Source': 'Internet Movie Database', 'Value'...",9.6,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN
595,Manchester by the Sea,2016,R,16 Dec 2016,137 min,Drama,Kenneth Lonergan,Kenneth Lonergan,"Casey Affleck, Michelle Williams, Kyle Chandler",A depressed uncle is asked to take care of his...,...,"[{'Source': 'Internet Movie Database', 'Value'...",9.6,7.8,"321,828",tt4034228,movie,"$47,695,371",NaN,True,NaN
822,Gravity,2013,PG-13,04 Oct 2013,91 min,"Drama, Sci-Fi, Thriller",Alfonso Cuarón,"Alfonso Cuarón, Jonás Cuarón","Sandra Bullock, George Clooney, Ed Harris","Dr Ryan Stone, an engineer on her first time o...",...,"[{'Source': 'Internet Movie Database', 'Value'...",9.6,7.7,"872,078",tt1454468,movie,"$274,092,705",NaN,True,NaN


In [75]:
import ast
def get_tomatoes(ratings_str):
    try:
        ratings_list = ast.literal_eval(ratings_str)

        for rating in ratings_list:
            if rating['Source'] == 'Rotten Tomatoes':
                # Extract percentage value and convert to float
                return float(rating['Value'].strip('%'))
        return np.nan
    except (ValueError, SyntaxError):
        return np.nan 

cleaned_scores_df['rotten_tomatoes_score'] = cleaned_scores_df['Ratings'].apply(get_tomatoes)

C:\Users\duvoe\AppData\Local\Temp\ipykernel_2208\1235251588.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scores_df['rotten_tomatoes_score'] = cleaned_scores_df['Ratings'].apply(get_tomatoes)


In [79]:
cleaned_scores_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error,rotten_tomatoes_score
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,10.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN,97.0
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,9.9,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN,98.0
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,9.6,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN,96.0
595,Manchester by the Sea,2016,R,16 Dec 2016,137 min,Drama,Kenneth Lonergan,Kenneth Lonergan,"Casey Affleck, Michelle Williams, Kyle Chandler",A depressed uncle is asked to take care of his...,...,9.6,7.8,"321,828",tt4034228,movie,"$47,695,371",NaN,True,NaN,96.0
822,Gravity,2013,PG-13,04 Oct 2013,91 min,"Drama, Sci-Fi, Thriller",Alfonso Cuarón,"Alfonso Cuarón, Jonás Cuarón","Sandra Bullock, George Clooney, Ed Harris","Dr Ryan Stone, an engineer on her first time o...",...,9.6,7.7,"872,078",tt1454468,movie,"$274,092,705",NaN,True,NaN,96.0


In [81]:
cleaned_scores_df.rename(columns={"rotten_tomatoes_score": "Rotten Tomatoes"}, inplace=True)
cleaned_scores_df.head()


C:\Users\duvoe\AppData\Local\Temp\ipykernel_2208\3414734695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scores_df.rename(columns={"rotten_tomatoes_score": "Rotten Tomatoes"}, inplace=True)


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error,Rotten Tomatoes
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,10.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN,97.0
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,9.9,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN,98.0
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,9.6,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN,96.0
595,Manchester by the Sea,2016,R,16 Dec 2016,137 min,Drama,Kenneth Lonergan,Kenneth Lonergan,"Casey Affleck, Michelle Williams, Kyle Chandler",A depressed uncle is asked to take care of his...,...,9.6,7.8,"321,828",tt4034228,movie,"$47,695,371",NaN,True,NaN,96.0
822,Gravity,2013,PG-13,04 Oct 2013,91 min,"Drama, Sci-Fi, Thriller",Alfonso Cuarón,"Alfonso Cuarón, Jonás Cuarón","Sandra Bullock, George Clooney, Ed Harris","Dr Ryan Stone, an engineer on her first time o...",...,9.6,7.7,"872,078",tt1454468,movie,"$274,092,705",NaN,True,NaN,96.0


In [83]:
cleaned_scores_df["Rotten Tomatoes"].fillna(value=0)

462      97.0
1384     98.0
819      96.0
595      96.0
822      96.0
         ... 
13887     0.0
13888     0.0
13889     0.0
13890     0.0
13891     0.0
Name: Rotten Tomatoes, Length: 12839, dtype: float64

In [85]:
cleaned_scores_df["Rotten Tomatoes"] = cleaned_scores_df["Rotten Tomatoes"]/10

C:\Users\duvoe\AppData\Local\Temp\ipykernel_2208\1042909904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scores_df["Rotten Tomatoes"] = cleaned_scores_df["Rotten Tomatoes"]/10


In [87]:
cleaned_scores_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Response,Error,Rotten Tomatoes
462,Boyhood,2014,R,15 Aug 2014,165 min,Drama,Richard Linklater,Richard Linklater,"Ellar Coltrane, Patricia Arquette, Ethan Hawke","The life of Mason, from early childhood to his...",...,10.0,7.9,"370,906",tt1065073,movie,"$25,352,281",NaN,True,NaN,9.7
1384,Moonlight,2016,R,18 Nov 2016,111 min,Drama,Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney","Mahershala Ali, Naomie Harris, Trevante Rhodes",A young African-American man grapples with his...,...,9.9,7.4,"336,819",tt4975722,movie,"$27,854,932",NaN,True,NaN,9.8
819,Roma,2018,R,14 Dec 2018,135 min,Drama,Alfonso Cuarón,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",A year in the life of a middle-class family's ...,...,9.6,7.7,"171,928",tt6155172,movie,NaN,NaN,True,NaN,9.6
595,Manchester by the Sea,2016,R,16 Dec 2016,137 min,Drama,Kenneth Lonergan,Kenneth Lonergan,"Casey Affleck, Michelle Williams, Kyle Chandler",A depressed uncle is asked to take care of his...,...,9.6,7.8,"321,828",tt4034228,movie,"$47,695,371",NaN,True,NaN,9.6
822,Gravity,2013,PG-13,04 Oct 2013,91 min,"Drama, Sci-Fi, Thriller",Alfonso Cuarón,"Alfonso Cuarón, Jonás Cuarón","Sandra Bullock, George Clooney, Ed Harris","Dr Ryan Stone, an engineer on her first time o...",...,9.6,7.7,"872,078",tt1454468,movie,"$274,092,705",NaN,True,NaN,9.6
